<a href="https://colab.research.google.com/github/ouriquco/Classificiation-Academic-Success/blob/summer_models/Academic_Success.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Reading in Data

In [3]:
train = pd.read_csv('train.csv', index_col = 'id')
test = pd.read_csv('test.csv', index_col = 'id')

# Data Preprocessing

In [6]:
#checking for NA
train.isnull().sum()
test.isnull().sum()

#checking for duplicates
train.duplicated().sum()
test.duplicated().sum()

features = test.columns

#convert categorical data to numeric
if train['Target'].dtype == 'object':
    le = LabelEncoder()
    train['Target'] = le.fit_transform(train['Target'])

#splitting into features and target
X_train = train[features]
y_train = train['Target']
X_test = test[features]

# Random Forest

# AdaBoost

# Gradient Boosting

# XGBoost

## CatBoost

# LightGBM